# Selector Group Chat
SelectorGroupChat implements a team where participants take turns broadcasting messages to all other members. A generative model (e.g., an LLM) selects the next speaker based on the shared context, enabling dynamic, context-aware collaboration.

Key features include:
- Model-based speaker selection
- Configurable participant roles and descriptions
- Prevention of consecutive turns by the same speaker (optional)
- Customizable selection prompting
- Customizable selection function to override the default model-based selection

SelectorGroupChat is a group chat similar to RoundRobinGroupChat, but with a model-based next speaker selection mechanism. When the team receives a task through run() or run_stream(), the following steps are executed:
1. The team analyzes the current conversation context, including the conversation history and participants’ name and description attributes, to determine the next speaker using a model. You can override the model by providing a custom selection function.
2. The team prompts the selected speaker agent to provide a response, which is then broadcasted to all other participants.
3. The termination condition is checked to determine if the conversation should end, if not, the process repeats from step 1.
4. When the conversation ends, the team returns the TaskResult containing the conversation history from this task.

Once the team finishes the task, the conversation context is kept within the team and all participants, so the next task can continue from the previous conversation context. You can reset the conversation context by calling reset().

## Load Azure Configurations

In [37]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [38]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Creating the Agents

In [62]:
from typing import Sequence
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from datetime import date


# Get the current date
current_date = date.today()

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=az_model_client,
    system_message=f"""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Multimodal Web Surfer: Searches for information in the internet on the weather advisory for the current date: {current_date} on the hiking location.
        Hiking Agent: Provides a hiking plan for the hiker based on the weather advisory.
        Product Agent: Recommends products from its file search tool.
        Writing Agent: Writes a document combining the hiking plan, product suggestions, and weather advisory.
        Critique Agent: Critiques the hiking plan, product agent, and writing agent and suggests improvements.
        
        
    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>
    """,
)

web_surfer_agent = MultimodalWebSurfer(
    name="MultimodalWebSurfer",
    model_client=az_model_client,
    description="A web surfer agent that searches for information on the internet.",
)

hiking_agent = AssistantAgent(
    "HikingAgent",
    description="A Hiking agent. Writes a hiking plan based on the weather advisory.",
    model_client=az_model_client,
    system_message="""
    You are a Hiking agent.
    You will create a hiking plan based on the weather advisory.
    After writing the hiking plan, you will pass it to the Critique agent for review.
    """,
)

critique_agent = AssistantAgent(
    "CritiqueAgent",
    description="A critique agent. Critiques the Hiking plan, product suggestions, and writing agent.",
    model_client=az_model_client,
    system_message="""
    You are a Critique agent.
    You need to ensure that the hiking plan is well thought out and provides constructive feedback.
    You need to ensure the multi-modal web surfer has provided the correct information on the weather
    You need to ensure that the product agent provides products from file search tool such as the CozyNights Sleeping Bag, BaseCamp Folding Table, and many more.
    You need to ensure that the writing agent has combined the hiking plan, the product suggestions, and weather advisory into a single document.
    You need to ensure feedback from user is incorporated and ask agents to make necessary changes.
    """,
)

# Define a tool to write the content to a file
async def write_to_file(content: str) -> str:
    # Write the content to a file
    with open("output/output.txt", "w") as file:
        file.write(content)

writing_agent = AssistantAgent(
    "WritingAgent",
    description="A Writing agent that combines the hiking plan, product suggestions, and weather advisory into a single document.",
    model_client=az_model_client,
    tools=[write_to_file],
    system_message="""
    You are a writing agent.
    You will combine the hiking plan, the product suggestions, and weather advisory into a single document.
    You will write the final output to a file.
    """,
)

# Create the user proxy agent.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

In [59]:
from autogen_core import CancellationToken
from autogen_ext.agents.openai import OpenAIAssistantAgent
from autogen_agentchat.messages import TextMessage
from openai import AsyncAzureOpenAI

cancellation_token = CancellationToken()

# Create AzureOpenAI client
client = AsyncAzureOpenAI(azure_endpoint=azure_openai_endpoint, 
                          api_version=azure_openai_api_version, 
                          api_key=azure_openai_key)

#### Create a vector store for file search ####
vector_store = await client.beta.vector_stores.create(name="Hiking Products")
# Specify the folder containing the files
folder_path = "../Data/products/"
# Get all file paths in the folder
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]
# Open file streams
file_streams = [open(path, "rb") for path in file_paths]
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = await client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)
# Close file streams
for file in file_streams:
    file.close()

#### Create an assistant with file search ####
product_agent = OpenAIAssistantAgent(
    name="ProductAgent",
    description="Suggestions for products",
    client=client,
    model=azure_openai_deployment,
    instructions="You will provide possible products for the hiker based on the file search tool such as the TrailMaster X4 Tent, CozyNights Sleeping Bag, and more.",
    tools=["file_search"],
    tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}}
)

In [ ]:
# Test the product agent
user_input = TextMessage(source="user", content="What is the price of the TrailMaster X4 Tent?")
response = await product_agent.on_messages([user_input], cancellation_token
)
print(response.chat_message.content)

## Creating the team
Let’s create the team with two termination conditions: 
- TextMentionTermination to end the conversation when the User Proxy sends “APPROVE”
- MaxMessageTermination to limit the conversation to avoid infinite loop.

In [63]:
text_mention_termination = TextMentionTermination("APPROVE")
max_messages_termination = MaxMessageTermination(max_messages=50)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, web_surfer_agent, hiking_agent, product_agent, writing_agent, critique_agent, user_proxy],
    model_client=az_model_client,
    termination_condition=termination,
)

## Specify the Task and Run the Team

In [ ]:
task = "I want to hike in the Grand Canyon, Arizona, USA."
# Use asyncio.run(...) if you are running this in a script.
response = await Console(team.run_stream(task=task))

## Printing whole response per agent source

In [ ]:
# ANSI escape code for bold text
bold_start = "\033[1m"
bold_end = "\033[0m"

# ANSI escape code for red text
red_start = "\033[31m"
red_end = "\033[0m"

for messages in response.messages:
    source = messages.source
    print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
    print(messages.content)

In [ ]:
for messages in response.messages:
    source = messages.source
    print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
    print(messages.content)

## Custom Selector Function
Often times we want better control over the selection process. To this end, we can set the selector_func argument with a custom selector function to override the default model-based selection. For instance, we want the Planning Agent to speak immediately after any specialized agent to check the progress.

In [ ]:
# This function determines whether the last message in the sequence was sent by the planning_agent. 
# If it wasn't, the function returns the name of the planning_agent; otherwise, it returns None.
# Returning None from the custom selector function will use the default model-based selection.
def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
await team.reset()

team = SelectorGroupChat(
    [planning_agent, web_surfer_agent, hiking_agent, product_agent, writing_agent, critique_agent, user_proxy],
    model_client=az_model_client,
    termination_condition=termination,
    selector_func=selector_func,
)

response = await Console(team.run_stream(task=task))

In [ ]:
for messages in response.messages:
    source = messages.source
    print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
    print(messages.content)

## Clean up the resources
Clean up the OpenAI Assistant Agent

In [72]:
# Clean up resources
await product_agent.delete_uploaded_files(cancellation_token)
await product_agent.delete_assistant(cancellation_token)